In [ ]:
import numpy as np
from optimal_levee_bandit import load_cost_curves, prune_candidates
from beta_dueling_bandit_analytical import run_beta_dueling_bandit_analytical

# 1. Load cost curves for your city
damage_file     = "Damage_cost_curves.tab"
protection_file = "Protection_cost_curves_high_estimate.tab"
city = "Halmstad"
heights, damage_costs, protection_costs = load_cost_curves(damage_file, protection_file, city)

# 2. Define analysis horizon and prune deterministic candidates
years_range = (2025, 2100)
n_years = years_range[1] - years_range[0] + 1
candidate_indices = prune_candidates(
    heights=heights,
    protection_costs=protection_costs,
    damage_costs=damage_costs,
    n_years=n_years
)
print("Candidate heights (m):", heights[candidate_indices])

# 3. Load posterior draws and mean sea level paths
pp = np.load("pp_inputs_halmsdad_pp_mixture_2025_2100.npz")
posterior_params = {
    "eta0": pp["eta0"],
    "eta1": pp["eta1"],
    "alpha0": pp["alpha0"],
    "xi": pp["xi"],
    "u": float(pp["u"]),  # threshold in cm
}
years_future   = pp["years_future"]
X_future_paths = pp["X_future_paths"]  # shape (M_pred, T_future) in cm

# 4. Run the analytical Beta-dueling bandit
#    - delta: misidentification probability target (e.g., 0.05 for 95% confidence)
#    - max_rounds: maximum number of simulated scenarios
#    - check_every: evaluate stopping rule every this many scenarios
rng = np.random.default_rng(42)
best_height, history = run_beta_dueling_bandit_analytical(
    heights=heights,
    damage_costs=damage_costs,
    protection_costs=protection_costs,
    candidate_indices=candidate_indices,
    years_all=years_future,
    X_pred_paths_cm=X_future_paths,
    posterior_params=posterior_params,
    years_range=years_range,
    delta=0.05,
    max_rounds=50_000,
    check_every=1_000,
    rng=rng,
    verbose=True,
)

print(f"Selected design height: {best_height:.2f} m")
print("Diagnostic history:", history)
